In [1]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)



In [2]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [3]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [4]:
print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [7]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [10]:
from langchain_community.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(model="gpt-3.5-turbo")

/opt/anaconda3/envs/jupyter/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
chain.invoke({"input": "What's the square of a triangle?"})

AIMessage(content='A triangle does not have a square. The square is a geometric shape with four equal sides and four right angles, whereas a triangle only has three sides.', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 52, 'total_tokens': 83}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d11f80f5-bd0e-47fe-8267-56b1cbaf02e6-0')

chat일 경우 크게 달라지는 것은 없고   
example_prompt 형식이 chat 형식으로 변경되는 것 밖에 없다. (promptTemplate -> ChatPromptTemplate)   
그에 맞게 fewShotPromptTemplate도 fewShotChatMessagePromptTemplate로 변경된다.

In [13]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

In [14]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [17]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_selector.select_examples({"input": "calculate"})

[{'input': '2+3', 'output': '5'}, {'input': '2+2', 'output': '4'}]

In [19]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    )
)

In [20]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [22]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [23]:
from langchain_community.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(model="gpt-3.5-turbo")

In [24]:
chain.invoke({"input" : "what is '1+1"})

AIMessage(content='1 + 1 is equal to 2.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 50, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5de2d9b-096d-4455-bcab-02673eacb26f-0')